In [1]:
import pandas as pd
from rdkit import Chem

In [2]:
df = pd.read_csv("data/processed/smiles_aggregated.csv")
base_df = df[["mol", "pIC50"]]
base_df = base_df.rename(columns={"mol": "SMILES"})
start_length = len(base_df)
base_df

,SMILES,pIC50
0,O1CC[C@@H](NC(=O)[C@@H](Cc2cc3cc(ccc3nc2N)-c2c...,9.154901
1,Fc1cc(cc(F)c1)C[C@H](NC(=O)[C@@H](N1CC[C@](NC(...,8.853872
2,S1(=O)(=O)N(c2cc(cc3c2n(cc3CC)CC1)C(=O)N[C@H](...,8.698970
3,S1(=O)(=O)C[C@@H](Cc2cc(O[C@H](COCC)C(F)(F)F)c...,8.698970
4,S1(=O)(=O)N(c2cc(cc3c2n(cc3CC)CC1)C(=O)N[C@H](...,8.698970
...,...,...
1514,Clc1cc2nc(n(c2cc1)C(CC(=O)NCC1CCOCC1)CC)N,3.000000
1515,Clc1cc2nc(n(c2cc1)C(CC(=O)NCc1ncccc1)CC)N,3.000000
1516,Brc1cc(ccc1)C1CC1C=1N=C(N)N(C)C(=O)C=1,2.953115
1517,O=C1N(C)C(=NC(=C1)C1CC1c1cc(ccc1)-c1ccccc1)N,2.733298


In [3]:
def Validate(smiles):
    return Chem.MolFromSmiles(smiles) != None

def DropInvalid(df):
    return df[df['SMILES'].map(lambda x: Validate(x))]

def ToCanonicalSmiles(df):
    df['SMILES'] = df['SMILES'].map(lambda x: Chem.MolToSmiles(Chem.MolFromSmiles(x)))
    return df

In [4]:
def ValidateSmiles(base_df):
    start_len = len(base_df)
    base_df = DropInvalid(base_df)
    invalid_len = start_len - len(base_df)
    ToCanonicalSmiles(base_df)
    base_df = base_df.drop_duplicates('SMILES')
    duplicates_len = start_len - invalid_len - len(base_df)
    return base_df, [start_len, invalid_len, duplicates_len]

In [5]:
# invalidate 2 molecules for testing
base_df.loc[0,'SMILES'] = 'Cc1ccccc1-c1ccc2nc(N)c(C[C@@H](C)C(=O)N[C@@HC(C)(C)C3)cc2c1'
base_df.loc[4,'SMILES'] = 'Cc1ccccc1-c1ccc2nc(N)c@@H](C)C(=O)N[C@@HC(C)(C)C3)cc2c1'
ValidateSmiles(base_df)

[18:39:28] SMILES Parse Error: syntax error while parsing: Cc1ccccc1-c1ccc2nc(N)c(C[C@@H](C)C(=O)N[C@@HC(C)(C)C3)cc2c1
[18:39:28] SMILES Parse Error: Failed parsing SMILES 'Cc1ccccc1-c1ccc2nc(N)c(C[C@@H](C)C(=O)N[C@@HC(C)(C)C3)cc2c1' for input: 'Cc1ccccc1-c1ccc2nc(N)c(C[C@@H](C)C(=O)N[C@@HC(C)(C)C3)cc2c1'
[18:39:28] SMILES Parse Error: syntax error while parsing: Cc1ccccc1-c1ccc2nc(N)c@@H](C)C(=O)N[C@@HC(C)(C)C3)cc2c1
[18:39:28] SMILES Parse Error: Failed parsing SMILES 'Cc1ccccc1-c1ccc2nc(N)c@@H](C)C(=O)N[C@@HC(C)(C)C3)cc2c1' for input: 'Cc1ccccc1-c1ccc2nc(N)c@@H](C)C(=O)N[C@@HC(C)(C)C3)cc2c1'
<ipython-input-3-9e0eeda1e357>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['SMILES'] = df['SMILES'].map(lambda x: Chem.MolToSmiles(Chem.MolFromSmi

(                                                 SMILES     pIC50
 1     CCCO[C@H]1C[NH2+][C@@H]([C@@H](O)[C@H](Cc2cc(F...  8.853872
 2     CCc1cn2c3c(cc(C(=O)N[C@@H](Cc4ccccc4)[C@H](O)C...  8.698970
 3     CCOC[C@@H](Oc1cc(C[C@@H]2CS(=O)(=O)C[C@H]([NH2...  8.698970
 5     CC(C)(C)c1cccc(C[NH2+][C@H]2CS(=O)C[C@@H](Cc3c...  8.698970
 6     CCCCCS(=O)(=O)C[C@@H](NC(=O)c1cccnc1)C(=O)N[C@...  8.698970
 ...                                                 ...       ...
 1514          CCC(CC(=O)NCC1CCOCC1)n1c(N)nc2cc(Cl)ccc21  3.000000
 1515          CCC(CC(=O)NCc1ccccn1)n1c(N)nc2cc(Cl)ccc21  3.000000
 1516                  Cn1c(N)nc(C2CC2c2cccc(Br)c2)cc1=O  2.953115
 1517           Cn1c(N)nc(C2CC2c2cccc(-c3ccccc3)c2)cc1=O  2.733298
 1518                  Nc1nc2cc(Cl)ccc2n1CCCC(=O)NCC1CC1  2.544546
 
 [1508 rows x 2 columns],
 [1519, 2, 9])